# Factorizations and other fun
Based on work by Andreas Noack

## Outline
 - Factorizations
 - Special matrix structures
 - Generic linear algebra

Before we get started, let's set up a linear system and use `LinearAlgebra` to bring in the factorizations and special matrix structures.

In [1]:
using LinearAlgebra
A = rand(3, 3)
x = fill(1, (3,))
b = A * x

3-element Vector{Float64}:
 1.5835617047539348
 1.5929101860507058
 1.364382157195217

## Factorizations

#### LU factorizations
In Julia we can perform an LU factorization
```julia
PA = LU
``` 
where `P` is a permutation matrix, `L` is lower triangular unit diagonal and `U` is upper triangular, using `lufact`.

Julia allows computing the LU factorization and defines a composite factorization type for storing it.

In [2]:
Alu = lu(A)

LU{Float64, Matrix{Float64}, Vector{Int64}}
L factor:
3×3 Matrix{Float64}:
 1.0       0.0       0.0
 0.354811  1.0       0.0
 0.446901  0.635176  1.0
U factor:
3×3 Matrix{Float64}:
 0.919236  0.341291  0.323035
 0.0       0.711329  0.319716
 0.0       0.0       0.00179221

In [3]:
typeof(Alu)

LU{Float64, Matrix{Float64}, Vector{Int64}}

The different parts of the factorization can be extracted by accessing their special properties

In [4]:
Alu.P

3×3 Matrix{Float64}:
 1.0  0.0  0.0
 0.0  1.0  0.0
 0.0  0.0  1.0

In [5]:
Alu.L

3×3 Matrix{Float64}:
 1.0       0.0       0.0
 0.354811  1.0       0.0
 0.446901  0.635176  1.0

In [6]:
Alu.U

3×3 Matrix{Float64}:
 0.919236  0.341291  0.323035
 0.0       0.711329  0.319716
 0.0       0.0       0.00179221

Julia can dispatch methods on factorization objects.

For example, we can solve the linear system using either the original matrix or the factorization object.

In [7]:
A\b

3-element Vector{Float64}:
 1.000000000000002
 1.0000000000000056
 0.9999999999999885

In [8]:
Alu\b

3-element Vector{Float64}:
 1.000000000000002
 1.0000000000000056
 0.9999999999999885

Similarly, we can calculate the determinant of `A` using either `A` or the factorization object

In [9]:
det(A) ≈ det(Alu)

true

#### QR factorizations

In Julia we can perform a QR factorization
```
A=QR
``` 

where `Q` is unitary/orthogonal and `R` is upper triangular, using `qrfact`. 

In [10]:
Aqr = qr(A)

LinearAlgebra.QRCompactWY{Float64, Matrix{Float64}, Matrix{Float64}}
Q factor:
3×3 LinearAlgebra.QRCompactWYQ{Float64, Matrix{Float64}, Matrix{Float64}}:
 -0.868544   0.460265  -0.183813
 -0.308169  -0.792007  -0.527026
 -0.388153  -0.4011     0.829732
R factor:
3×3 Matrix{Float64}:
 -1.05836  -0.78753   -0.549974
  0.0      -0.744602  -0.33539
  0.0       0.0        0.00148706

Similarly to the LU factorization, the matrices `Q` and `R` can be extracted from the QR factorization object via

In [11]:
Aqr.Q

3×3 LinearAlgebra.QRCompactWYQ{Float64, Matrix{Float64}, Matrix{Float64}}:
 -0.868544   0.460265  -0.183813
 -0.308169  -0.792007  -0.527026
 -0.388153  -0.4011     0.829732

In [12]:
Aqr.R

3×3 Matrix{Float64}:
 -1.05836  -0.78753   -0.549974
  0.0      -0.744602  -0.33539
  0.0       0.0        0.00148706

#### Eigendecompositions

The results from eigendecompositions, singular value decompositions, Hessenberg factorizations, and Schur decompositions are all stored in `Factorization` types.

The eigendecomposition can be computed

In [13]:
Asym = A + A'
AsymEig = eigen(Asym)

Eigen{Float64, Float64, Matrix{Float64}, Vector{Float64}}
values:
3-element Vector{Float64}:
 -0.010267013741392272
  1.1327664253099456
  3.0792830466185164
vectors:
3×3 Matrix{Float64}:
 -0.17517    0.7673    -0.616901
 -0.467474  -0.616273  -0.633779
  0.866478  -0.177366  -0.466645

The values and the vectors can be extracted from the Eigen type by special indexing

In [14]:
AsymEig.values

3-element Vector{Float64}:
 -0.010267013741392272
  1.1327664253099456
  3.0792830466185164

In [15]:
AsymEig.vectors

3×3 Matrix{Float64}:
 -0.17517    0.7673    -0.616901
 -0.467474  -0.616273  -0.633779
  0.866478  -0.177366  -0.466645

Once again, when the factorization is stored in a type, we can dispatch on it and write specialized methods that exploit the properties of the factorization, e.g. that $A^{-1}=(V\Lambda V^{-1})^{-1}=V\Lambda^{-1}V^{-1}$.

In [17]:
inv(AsymEig)*Asym

3×3 Matrix{Float64}:
  1.0           4.08562e-14  -6.92779e-14
  6.39488e-14   1.0          -1.91847e-13
 -1.42109e-13  -2.13163e-13   1.0

## Special matrix structures
Matrix structure is very important in linear algebra. To see *how* important it is, let's work with a larger linear system

In [18]:
n = 1000
A = randn(n,n);

Julia can often infer special matrix structure

In [19]:
Asym = A + A'
issymmetric(Asym)

true

but sometimes floating point error might get in the way.

In [20]:
Asym_noisy = copy(Asym)
Asym_noisy[1,2] += 5eps()

-0.3350106546165601

In [21]:
issymmetric(Asym_noisy)

false

Luckily we can declare structure explicitly with, for example, `Diagonal`, `Triangular`, `Symmetric`, `Hermitian`, `Tridiagonal` and `SymTridiagonal`.

In [22]:
Asym_explicit = Symmetric(Asym_noisy);

Let's compare how long it takes Julia to compute the eigenvalues of `Asym`, `Asym_noisy`, and `Asym_explicit`

In [23]:
@time eigvals(Asym);

  0.249607 seconds (268.44 k allocations: 25.881 MiB, 2.50% gc time, 77.97% compilation time)


In [24]:
@time eigvals(Asym_noisy);

  0.337595 seconds (14 allocations: 7.928 MiB)


In [25]:
@time eigvals(Asym_explicit);

  0.096802 seconds (69.63 k allocations: 12.734 MiB, 43.79% compilation time)


In this example, using `Symmetric()` on `Asym_noisy` made our calculations about `5x` more efficient :)

#### A big problem
Using the `Tridiagonal` and `SymTridiagonal` types to store tridiagonal matrices makes it possible to work with potentially very large tridiagonal problems. The following problem would not be possible to solve on a laptop if the matrix had to be stored as a (dense) `Matrix` type.

In [26]:
n = 1_000_000;
A = SymTridiagonal(randn(n), randn(n-1));
@time eigmax(A)

  0.730921 seconds (906.33 k allocations: 238.119 MiB, 1.81% gc time, 30.45% compilation time)


6.393506601982381

## Generic linear algebra
The usual way of adding support for numerical linear algebra is by wrapping BLAS and LAPACK subroutines. For matrices with elements of `Float32`, `Float64`, `Complex{Float32}` or `Complex{Float64}` this is also what Julia does.

However, Julia also supports generic linear algebra, allowing you to, for example, work with matrices and vectors of rational numbers.

#### Rational numbers
Julia has rational numbers built in. To construct a rational number, use double forward slashes:

In [27]:
1//2

1//2

#### Example: Rational linear system of equations
The following example shows how linear system of equations with rational elements can be solved without promoting to floating point element types. Overflow can easily become a problem when working with rational numbers so we use `BigInt`s.

In [28]:
Arational = Matrix{Rational{BigInt}}(rand(1:10, 3, 3))/10

3×3 Matrix{Rational{BigInt}}:
 7//10  2//5   4//5
 2//5   4//5   1//5
 2//5   3//10  3//5

In [29]:
x = fill(1, 3)
b = Arational*x

3-element Vector{Rational{BigInt}}:
 19//10
  7//5
 13//10

In [30]:
Arational\b

3-element Vector{Rational{BigInt}}:
 1//1
 1//1
 1//1

In [31]:
lu(Arational)

LU{Rational{BigInt}, Matrix{Rational{BigInt}}, Vector{Int64}}
L factor:
3×3 Matrix{Rational{BigInt}}:
 1//1  0//1  0//1
 4//7  1//1  0//1
 4//7  1//8  1//1
U factor:
3×3 Matrix{Rational{BigInt}}:
 7//10  2//5   4//5
 0//1   4//7  -9//35
 0//1   0//1   7//40

### Exercises

#### 11.1
What are the eigenvalues of matrix A?

```
A =
[
 140   97   74  168  131
  97  106   89  131   36
  74   89  152  144   71
 168  131  144   54  142
 131   36   71  142   36
]
```
and assign it a variable `A_eigv`

In [1]:
using LinearAlgebra

In [17]:
A = [
 140   97   74  168  131
  97  106   89  131   36
  74   89  152  144   71
 168  131  144   54  142
 131   36   71  142   36
]
A_eigv = eigen(A).values

5-element Vector{Float64}:
 -128.49322764802147
  -55.88778455305702
   42.752167279318854
   87.161114775145
  542.4677301466139

In [18]:
@assert A_eigv ==  [-128.49322764802145, -55.887784553056875, 42.7521672793189, 87.16111477514521, 542.4677301466143]

LoadError: AssertionError: A_eigv == [-128.49322764802145, -55.887784553056875, 42.7521672793189, 87.16111477514521, 542.4677301466143]

#### 11.2 
Create a `Diagonal` matrix from the eigenvalues of `A`.

In [20]:
A_diag = Diagonal(A_eigv)

5×5 Diagonal{Float64, Vector{Float64}}:
 -128.493     ⋅        ⋅        ⋅         ⋅ 
     ⋅     -55.8878    ⋅        ⋅         ⋅ 
     ⋅        ⋅      42.7522    ⋅         ⋅ 
     ⋅        ⋅        ⋅      87.1611     ⋅ 
     ⋅        ⋅        ⋅        ⋅      542.468

In [21]:
@assert A_diag ==  [-128.493    0.0      0.0      0.0       0.0;
    0.0    -55.8878   0.0      0.0       0.0;
    0.0      0.0     42.7522   0.0       0.0;
    0.0      0.0      0.0     87.1611    0.0;
    0.0 0.0      0.0      0.0     542.468]

LoadError: AssertionError: A_diag == [-128.493 0.0 0.0 0.0 0.0; 0.0 -55.8878 0.0 0.0 0.0; 0.0 0.0 42.7522 0.0 0.0; 0.0 0.0 0.0 87.1611 0.0; 0.0 0.0 0.0 0.0 542.468]

#### 11.3 
Create a `LowerTriangular` matrix from `A` and store it in `A_lowertri`

In [22]:
A_lowertri = LowerTriangular(A)

5×5 LowerTriangular{Int64, Matrix{Int64}}:
 140    ⋅    ⋅    ⋅   ⋅
  97  106    ⋅    ⋅   ⋅
  74   89  152    ⋅   ⋅
 168  131  144   54   ⋅
 131   36   71  142  36

In [23]:
@assert A_lowertri ==  [140    0    0    0   0;
  97  106    0    0   0;
  74   89  152    0   0;
 168  131  144   54   0;
 131   36   71  142  36]